In [1]:
import pandas as pd


%load_ext lab_black

In [2]:
df = pd.read_csv("purchase_data.csv")

In [3]:
# player count

total_players = pd.DataFrame({df.SN.nunique()})
total_players.columns = ["Total Players"]
total_players

,Total Players
0,576


In [4]:
# purchasing Analysis (Total)

purchasing_analysis = pd.DataFrame(
    {
        "Number of Unique Items": [df["Item ID"].nunique()],
        "Average Price": "$" + format(df["Price"].mean(), ".2f"),
        "Number of Purchases": [df["SN"].count()],
        "Total Revenue": "$" + format(df["Price"].sum(), ".2f"),
    }
)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [5]:
# Gender Demographics

# Column index 0
g_count = df.groupby(df["Gender"])["SN"].nunique().sort_values(ascending=False)


# Column Index 1
total_count = df["SN"].nunique()


def gender_per(e):
    return e / total_count


percent_of_players = [gender_per(e) for e in g_count]


def per_stylized(e):
    return str(format(e * 100, ".2f")) + "%"


per_players = [per_stylized(e) for e in percent_of_players]


gender_dem = pd.DataFrame({"Total Count": g_count, "Percent of Players": per_players})
gender_dem

,Total Count,Percent of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
# Purchase Analysis (GENDER)

purchase_count = df.groupby(df["Gender"])["Purchase ID"].count()

g_avg_price = df.groupby(df["Gender"])["Price"].mean()

g_total_purchase = df.groupby(df["Gender"])["Price"].sum()

g_avg_total_purchase = (g_total_purchase / g_count).map("${:,.2f}".format)


gen_pur = pd.DataFrame(
    {
        "Purchase Count": purchase_count,
        "Average Purchase Price": g_avg_price,
        "Total Purchase Value": g_total_purchase,
        "Avg Total Purchase per Person": g_avg_total_purchase,
    }
)

gen_pur

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,$4.47
Male,652,3.017853,1967.64,$4.07
Other / Non-Disclosed,15,3.346000,50.19,$4.56


In [7]:
# Age Demographics
# needs work

df_unique = df[["Age", "SN"]].drop_duplicates()

df_unique["age_counts"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)


age_count = df_unique["age_counts"].value_counts()


def players(e):
    return e / total_count


per_players = [str(format(players(e) * 100, ".2f")) + "%" for e in age_count]


df_age = pd.DataFrame({"Total Count": age_count, "Percent of Players": per_players})
df_age

,Total Count,Percent of Players
20-24,258,44.79%
15-19,107,18.58%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
10-14,22,3.82%
<10,17,2.95%
40+,12,2.08%


In [10]:
# Purchasing Analaysis

age_counts = df["Purchase ID"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)

age_counts

0      20-24
1        40+
2      20-24
3      20-24
4      20-24
       ...  
775    20-24
776    20-24
777    20-24
778      <10
779    20-24
Name: Age, Length: 780, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [12]:
# Top Spenders
# formatting


purchase_count = df.groupby(["SN"]).agg({"Price": ["sum", "count", "mean"]})

purchase_count.columns = map("_".join, purchase_count)

purchase_count = purchase_count.rename(
    columns={
        "Price_count": "Purchase Count",
        "Price_mean": "Average Purchase Price",
        "Price_sum": "Total Purchase Value",
    }
).sort_values(by="Total Purchase Value", ascending=False)

purchase_count

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,18.96,5,3.792000
Idastidru52,15.45,4,3.862500
Chamjask73,13.83,3,4.610000
Iral74,13.62,4,3.405000
Iskadarya95,13.10,3,4.366667
...,...,...,...
Ililsasya43,1.02,1,1.020000
Irilis75,1.02,1,1.020000
Aidai61,1.01,1,1.010000


In [13]:
# Most Popular Items


most_pop = df.groupby(["Item ID", "Item Name", "Price"]).agg(
    {"Price": ["sum", "count"]}
)


most_pop.columns = map("_".join, most_pop)

most_pop = most_pop.rename(
    columns={"Price_count": "Purchase Count", "Price_sum": "Total Purchase Value",}
).sort_values(by="Purchase Count", ascending=False)


most_pop

,,,Total Purchase Value,Purchase Count
Item ID,Item Name,Price,,
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,50.76,12
145,Fiery Glass Crusader,4.58,41.22,9
108,"Extraction, Quickblade Of Trembling Hands",3.53,31.77,9
82,Nirvana,4.90,44.10,9
19,"Pursuit, Cudgel of Necromancy",1.02,8.16,8
...,...,...,...,...
104,Gladiator's Glaive,1.93,1.93,1
23,Crucifer,1.99,1.99,1
180,Stormcaller,3.36,3.36,1
